# 🧹 Limpieza y preparación de datos
En este notebook se limpian y transforman los datos históricos de los Mundiales y los datos del fixture del Mundial 2022. Este paso es esencial para preparar los conjuntos de datos que alimentarán el modelo de predicción y permitirán simular los partidos del torneo.

In [183]:
import pandas as pd

## 1.Carga de datos históricos y del fixture
Cargamos el archivo fifa_worldcup_historical_data.csv, que contiene los resultados de todos los partidos jugados en los mundiales desde 1930 hasta 2018. El archivo fifa_worldcup_fixture.csv contiene los enfrentamientos del torneo 2022. 

Estos datasets fueron generados previamente en el script de web scraping (get_matches).

In [184]:
df_data_historica = pd.read_csv('fifa_worldcup_historical_data.csv')
df_fixture = pd.read_csv('fifa_worldcup_fixture.csv')

### 1.1 Limpieza de df_fixture
Los nombres de los equipos contienen caractéres en blanco. Hay que arreglarlo

In [185]:
df_fixture['home'] = df_fixture['home'].str.strip()
df_fixture['away'] = df_fixture['away'].str.strip()

### 1.2 Limpieza de df_data_historica
En el mundial de 1938, el partido Suecia - Austria no se disputó por la anexión de Austria a Alemania. Este valor hay que eliminarlo

In [186]:
df_data_historica[df_data_historica['home'].str.contains('Sweden') &
                  df_data_historica['away'].str.contains('Austria')]

,home,score,away,year
37,Sweden,w/o,Austria,1938


In [187]:
df_data_historica.drop(index = 37, inplace = True)

Los partidos que se decidieron por penaltis, contienen en score (a.e.t). Esto ensucia los datos, por lo que eliminamos todo lo que no sea digito - digito

In [188]:
df_data_historica['score'] = df_data_historica['score'].str.replace(r'[^\d–]', '', regex = True)

Eliminamos posibles caractéres en blanco

In [189]:
df_data_historica['home'] = df_data_historica['home'].str.strip()
df_data_historica['away'] = df_data_historica['away'].str.strip()

Descomponemos la columna score (que contiene resultados como '3–1') en dos columnas numéricas: HomeGoals y AwayGoals. Esto nos permitirá más adelante ajustar modelos estadísticos sobre los goles marcados por cada equipo.

In [190]:
df_data_historica[['HomeGoals', 'AwayGoals']] = df_data_historica['score'].str.split('–', expand = True)

In [191]:
df_data_historica.drop('score', axis = 1, inplace = True)

Renombramos columnas

In [192]:
df_data_historica.rename(columns={'home':'HomeTeam', 'away':'AwayTeam', 'year':'Year'}, inplace=True)
df_data_historica

,HomeTeam,AwayTeam,Year,HomeGoals,AwayGoals
0,France,Mexico,1930,4,1
1,Argentina,France,1930,1,0
2,Chile,Mexico,1930,3,0
3,Chile,France,1930,1,0
4,Argentina,Mexico,1930,6,3
...,...,...,...,...,...
860,Russia,Croatia,2018,2,2
861,France,Belgium,2018,1,0
862,Croatia,England,2018,2,1
863,Belgium,England,2018,2,0


Verificamos que los valores son del tipo correcto

In [193]:
df_data_historica.dtypes

HomeTeam     object
AwayTeam     object
Year          int64
HomeGoals    object
AwayGoals    object
dtype: object

Queremos que los goles sean variables numéricas

In [194]:
df_data_historica = df_data_historica.astype({'HomeGoals': int, 'AwayGoals': int})
df_data_historica.dtypes

HomeTeam     object
AwayTeam     object
Year          int64
HomeGoals     int64
AwayGoals     int64
dtype: object

# 2. Exportar dataframes limpios

Exportamos los datasets en formato .csv para su posterior uso en el modelo predictivo de Poisson que simulará todo el Mundial.

In [195]:
df_data_historica.to_csv('clean_fifa_worldcup_historical_data.csv', index = False)
df_fixture.to_csv('clean_fifa_worldcup_fixture.csv', index = False)